## Project Data Wrangling - We rate dogs

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Gathering Data](#gather)
- [Assessing Data](#assess)
- [Cleaning Data](#clean)
- [Analyzing Data](#analyze)




In [33]:
import tweepy
import pandas as pd
import numpy as np
import requests
import os
import json
import re
# use module dotenv to manage API keys and secrets
%load_ext dotenv
%dotenv

consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')
access_token = os.environ.get('ACCESS_TOKEN')
access_secret = os.environ.get('ACCESS_SECRET')
tsv_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
tsv_filename = "image-predictions.tsv"
json_filename = "tweets_json.txt"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


<a id='gather'></a>
### Gathering Data
We gather data from 3 several sources:
1. enhanced Twitter archive: a csv file 'twitter-archive-enhanced.csv' (data stored in `df_tweets_raw`)
2. Additional Data via the Twitter API
3. Image Predictions File

##### 1.) Gathering Data: enhanced twitter archive
Let's start with getting the data from twitter archive file and take a look at a few records:


In [31]:
df_tweets_raw = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_raw.sample(4)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
735,781163403222056960,NaN,NaN,2016-09-28 16:07:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...","We normally don't rate lobsters, but this one ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/781163403...,10,10,None,None,None,None,None
585,800018252395122689,NaN,NaN,2016-11-19 16:49:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo doin a struggle. 11/10 much det...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/800018252...,11,10,None,doggo,None,None,None
1383,700847567345688576,NaN,NaN,2016-02-20 01:00:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Crouton. He's a Galapagos Boonwiddle. Has...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/700847567...,10,10,Crouton,None,None,None,None
1646,683834909291606017,NaN,NaN,2016-01-04 02:18:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we see a faulty pupper. Might need to rep...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683834909...,9,10,None,None,None,pupper,None


##### 2.) Gathering Data: Additional data via twitter api
Get data via twitter api if not already downloaded and store it in file "tweets_json.txt":

In [ ]:
if not (os.path.exists(json_filename)):
    # get tweets via api and store in json list:
    json_list = []
    i = 1
    for tweet_id in df_tweets_raw.tweet_id:
        try:
            #print progress:
            if (i % 100 == 0):
                print("tweets fetched: {}".format(i))
            i = i + 1
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json_list.append(tweet._json)
        except tweepy.TweepError as err:
            print("Error for tweet {}, error: {}".format(tweet_id, err))
            pass
            
    #store json list in json file:
    with open(json_filename, 'w') as f:
        for json_tweet in json_list:
            json.dump(json_tweet, f)
            f.write("\n")
    

Error for tweet 888202515573088257, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 873697596434513921, error: [{'code': 144, 'message': 'No status found with that ID.'}]
tweets fetched: 100
Error for tweet 872668790621863937, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 872261713294495745, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 869988702071779329, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 866816280283807744, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 861769973181624320, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 856602993587888130, error: [{'code': 144, 'message': 'No status found with that ID.'}]
tweets fetched: 200
Error for tweet 851953902622658560, error: [{'code': 144, 'message': 'No status found with that ID.'}]
Error for tweet 84545907679661670

Rate limit reached. Sleeping for: 541


Error for tweet 758740312047005698, error: Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
Error for tweet 754011816964026368, error: [{'code': 144, 'message': 'No status found with that ID.'}]
tweets fetched: 1000
tweets fetched: 1100
tweets fetched: 1200
tweets fetched: 1300
tweets fetched: 1400
tweets fetched: 1500
tweets fetched: 1600
tweets fetched: 1700
Error for tweet 680055455951884288, error: [{'code': 144, 'message': 'No status found with that ID.'}]
tweets fetched: 1800


Rate limit reached. Sleeping for: 585


In [35]:
#read json file into data frame
df_json_tweets= pd.read_json(json_filename, lines=True)
df_json_tweets.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2,2017-07-31 00:18:03+00:00,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
3,2017-07-30 15:58:51+00:00,891689557279858688,891689557279858688,This is Darla. She commenced a snooze mid meal...,False,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
4,2017-07-29 16:00:24+00:00,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,False,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN


##### 3.) Gathering Data: Image Prediction file
Get file from internet if not already downloaded and show a few records

In [39]:
if not (os.path.exists(tsv_filename)):
    r = requests.get(tsv_url, allow_redirects=True)
    with open(tsv_filename, 'wb') as f:
            f.write(r.content)
df_img_pred = pd.read_csv(tsv_filename, sep="\t")
df_img_pred.head(10)
df_img_pred[df_img_pred.p1_dog == False]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,0.085547,False,bookcase,0.079480,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,0.015250,False,great_grey_owl,0.013207,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2026,882045870035918850,https://pbs.twimg.com/media/DD2oCl2WAAEI_4a.jpg,1,web_site,0.949591,False,dhole,0.017326,False,golden_retriever,0.006941,True
2046,886680336477933568,https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg,1,convertible,0.738995,False,sports_car,0.139952,False,car_wheel,0.044173,False
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False,tow_truck,0.029175,False,shopping_cart,0.026321,False
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False


<a id='assess'></a>
## Assessing Data

### 1.) Advanced Twitter Archive
**Quality:** 
- `name` column: some entries seem to be regular words rather than names (issue type: validity, visually explored)
- `rating_denominator` column: several entries are <> 10, indicating invalid rating values (issue type: validity, visually explored)
- `timestamp` column: data type is string, more appropriate format would be python's datetime format (issue type: validity, programmatically explored)
- several columns (`name`, `puppo`, etc.): NaN is string "None", more appropriate would be entry `np.nan`. (issue type: validity, visually explored)
- archive contains retweets which should not be included in the analysis.

**Tidiness:**
- columns `doggo`, `floofer`, `puppo` and `pupper` with mutual exclusive categorical data should be reduced to one column. 
- several columns are not being used in the analysis.

### 2.) Additional data Twitter API
**Quality:**
- additional data couldn't be fetched for all tweets from the archive (issue type: completeness)
- several columns are not being used in the analysis.

### 3.) Image prediction file
**Quality:**
- some tweets from the archive do not have a corresponding entry in the prediction file (issue type completeness)
- some predictions do not correctly assess the image's motif.

### Overall
**Tidiness:**
- Because of the 1:1:1 relationships between the records of the 3 tables, all data frames should be merged into a single table




In [6]:
df_tweets_raw.isnull().sum() / df_tweets_raw.shape[0]

tweet_id                      0.000000
in_reply_to_status_id         0.966893
in_reply_to_user_id           0.966893
timestamp                     0.000000
source                        0.000000
text                          0.000000
retweeted_status_id           0.923175
retweeted_status_user_id      0.923175
retweeted_status_timestamp    0.923175
expanded_urls                 0.025042
rating_numerator              0.000000
rating_denominator            0.000000
name                          0.000000
doggo                         0.000000
floofer                       0.000000
pupper                        0.000000
puppo                         0.000000
dtype: float64

In [15]:
df_tweets_raw['doggo'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2351    False
2352    False
2353    False
2354    False
2355    False
Name: doggo, Length: 2356, dtype: bool

In [18]:
df_tweets_raw['doggo'].replace('None', np.nan, inplace=True)

In [27]:
df_tweets_raw[df_tweets_raw.doggo.notna()].doggo

array(['doggo'], dtype=object)

<a id="clean"></a>
## Cleaning Data
### 1.) Twitter archive



In [41]:
#make copy to work with while cleaning the data
df_tweets_clean = df_tweets_raw.copy()

### Quality
##### Issue: Some names in the name column aren't actually names but regular words. 
##### Define
Issue is probably due to a naive assumptions in parsing process of the tweet's text ("This is *dogname*"). So, my solution is to create
a new name column and extract only valid names from the name column. That is, only names with more than one letter starting with upper case.
##### Code

In [42]:
df_tweets_clean['name_extract']=''
df_tweets_clean['name_extract']= df_tweets_clean.name.str.extract(r"^([A-Z]\w+)")

##### Test

In [43]:
# check to see if only regluar words are left and haven't been extracted from the original name column
df_tweets_clean[df_tweets_clean.name_extract != df_tweets_clean.name]['name'].value_counts()

a               55
the              8
an               7
very             5
quite            4
one              4
just             4
getting          2
mad              2
not              2
actually         2
O                1
incredibly       1
light            1
space            1
infuriating      1
this             1
by               1
old              1
his              1
my               1
such             1
unacceptable     1
life             1
all              1
officially       1
Name: name, dtype: int64

In [62]:
# replace contents of name and drop temporary column name_extract and test again
df_tweets_clean.name = df_tweets_clean.name_extract
df_tweets_clean.drop('name_extract', axis=1, inplace = True)
df_tweets_clean.name.value_counts()

None         728
Charlie       12
Cooper        11
Oliver        11
Lucy          11
            ... 
Fletcher       1
Tommy          1
Lucky          1
Bonaparte      1
Ralphy         1
Name: name, Length: 929, dtype: int64

##### Issue: `rating_denominator` column: several entries are <> 10, indicating invalid rating values.
##### Define
We are making the very rigourous assumption that only ratings with denominator = 10 are valid ratings. So, we delete all rows with denominators <> 10.
##### Code

In [57]:
invalid_ratings = df_tweets_clean[df_tweets_clean.rating_denominator != 10].index
df_tweets_clean.drop(invalid_ratings, inplace =True)

##### Test

In [59]:
# value counts should only show entries for value 10
df_tweets_clean.rating_denominator.value_counts()

10    2333
Name: rating_denominator, dtype: int64

##### Issue: `timestamp` column: data type is string
##### Define
convert datatype of `timestamp` to datetime
##### Code

In [64]:
df_tweets_clean.timestamp = pd.to_datetime(df_tweets_clean.timestamp)

##### Test

In [65]:
df_tweets_clean.timestamp.head(3)

0   2017-08-01 16:23:56+00:00
1   2017-08-01 00:17:27+00:00
2   2017-07-31 00:18:03+00:00
Name: timestamp, dtype: datetime64[ns, UTC]

##### Issue: several columns (`name`, `puppo`, etc.): NaN is string "None", more appropriate would be entry `np.nan`.
##### Define
convert string value "None" to `np.nan`
##### Code

In [94]:
df_tweets_clean = df_tweets_clean[['name','puppo', 'pupper', 'floofer', 'doggo']].replace('None', np.nan, regex=True)

TypeError: 'NoneType' object is not subscriptable

In [89]:
df_tweets_clean.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,name_extract
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,Phineas
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,Tilly
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,Archie
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,Darla
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,Franklin
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,Jax
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40+00:00,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,Zoey
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None,Cassie
